In [ ]:
import os
import requests
import json
import time
import boto3
import io
import concurrent.futures
import dotenv

# Cargar las variables de entorno desde el archivo .env
dotenv.load_dotenv()

# Definición de las variables globales
TMDB_KEY = os.getenv('API_KEY')
MOVIES_URL = "https://api.themoviedb.org/3/movie"
BUCKET_NAME = os.getenv('BUCKET_NAME_E')
session = boto3.Session()
s3 = session.client('s3')

def get_movie_detail(movie_id):
    try:
        response = requests.get(MOVIES_URL + f"/{movie_id}?api_key={TMDB_KEY}&language=en-US", timeout=10)
        if response.status_code == 200:
            print(f"Descargado id: {movie_id}")
            return response.json()
        else:
            print(f"Error id: {movie_id} - Status: {response.status_code}")
            return None
    except Exception as e:
        print(f"Excepción id: {movie_id} - {e}")
        return None

try:
    j = 1  # Punto de inicio para la descarga de películas

    while j <= 1_000_000:
        ids_lote = [j + i for i in range(5_000) if (j + i) <= 1_000_000]
        movies = []
        errores = 0
        print(f"\nDescargando películas desde el id {j} hasta {j+len(ids_lote)-1}...")

        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            future_to_id = {executor.submit(get_movie_detail, movie_id): movie_id for movie_id in ids_lote}
            for future in concurrent.futures.as_completed(future_to_id):
                movie_id = future_to_id[future]
                data = future.result()
                if data:
                    movies.append(data)
                else:
                    errores += 1

        file_name = f"movies{j}.json"
        print(f"\nGuardando lote en {file_name} con {len(movies)} películas. Errores en el batch: {errores}")

        if len(movies) > 0:
            buffer = io.BytesIO()
            buffer.write(json.dumps(movies, ensure_ascii=False, indent=4).encode('utf-8'))
            buffer.seek(0)
            try:
                s3.put_object(Bucket=BUCKET_NAME, Key=file_name, Body=buffer)
                print(f"Archivo {file_name} subido a S3 bucket {BUCKET_NAME}")
            except Exception as e:
                print(f"Error al subir {file_name} a S3:", e)
        else:
            print(f"No se subió {file_name} porque el lote está vacío.")

        if j >= 1_000_000:
            print("Fin de la descarga.")
            break

        j += 5_000

except Exception as e:
    print("Error global inesperado:", e)